# 75.70 Trabajo Práctico 1

## Imports y seteo de semillas para randoms

Se importan numpy, tensorflow, keras y sklearn. 

Se setean las semillas de randoms de numpy y tensorflow para tener consistencia entre corridas.

Seteamos constante con ubicacion del .csv.

In [0]:
import numpy as np
np.random.seed(91218) # Set np seed for consistent results across runs

import tensorflow as tf
tf.set_random_seed(91218)

from tensorflow import keras
from sklearn.model_selection import train_test_split

CSVPATH = '/content/Admission_Predict_Ver1.1.csv'

## Extracción y procesamiento de datos

Se definen las funciones para extracción y procesamiento del set de datos.

Se descarta la columna de IDs y se normalizan el resto de las columnas.

Creamos función para categorizar los datos de entrada segun la salida que tienen (Para la segunda red).

In [0]:
def categorize_admition(chance):
  if chance < 0.5:
    return 0 #'very_bad'
  elif chance < 0.65:
    return 1 #'bad'
  elif chance < 0.8:
    return 2 #'medium'
  elif chance < 0.9:
    return 3 #'good'
  else:
    return 4 #'very_good'

def process_row(row, categorize=False):
  return [
      float(row[1])/340.0,
      float(row[2])/120.0,
      float(row[3])/5.0,
      float(row[4])/5.0,
      float(row[5])/5.0,
      float(row[6])/10.0,
      float(row[7]),
      categorize_admition(float(row[8])) if categorize else float(row[8])
  ]
  
def extract_data(csvfile, categorize):
  rows = csvfile.read().splitlines()
  
  # Remove headaer and parse rows
  rows = rows[1:]
  data = [row.split(',') for row in rows]
  data = [process_row(row, categorize) for row in data]
  
  return data


## Formas de entrenamiento

Definimos funciones para entrenar. Puede ser utilizando los primeros n elementos del set, utilizando elementos random o los n primeros elementos usando categorias.

In [0]:
def fixed_train(x, y):
  max_x = int(len(x) * 0.67) # 33% used for test
  max_y = int(len(y) * 0.67) # 33% used for test
  return x[:max_x], x[max_x:], y[:max_y], y[max_y:]


def random_train(x, y):
  return train_test_split(x, y, test_size=0.33, random_state=43)


def categorical_train(x, y):
  x_train, x_test, y_train, y_test = fixed_train(x, y)
  y_train = keras.utils.to_categorical(y_train, num_classes=5)
  y_test = keras.utils.to_categorical(y_test, num_classes=5)
  return x_train, x_test, y_train, y_test

## Carga de datos

Se cargan los datos sin categorizar y categorizados.

Se toman las primeras 8 columnas como features y la ultima como output.

In [0]:
csv = open(CSVPATH, 'r')
normal_data = extract_data(csv, False)
categorized_data = extract_data(csv, True)
csv.close()

dataset = np.array(data)

x = dataset[:, 0:7]
y = dataset[:, 7]

## Entrenamiento de la primera red

Se crea la primera red usando datos sin categorizar. Como funcion de loss se utiliza el error cuadrático medio

In [83]:
#X_train, X_test, y_train, y_test = random_train(x, y)
X_train, X_test, y_train, y_test = fixed_train(x, y)
#X_train, X_test, y_train, y_test = categorical_train(x, y)


model_1 = keras.Sequential([
    keras.layers.Dense(1, input_dim=7, activation='relu'),
    keras.layers.Dense(1, activation='softmax')
])

model_1.compile(optimizer='adam', 
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

model_1.fit(X_train, y_train, epochs=10, batch_size=50, shuffle=False)
print(model_1.summary())

Epoch 1/10
335/335 [==============================] - 0s 1ms/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 2/10
335/335 [==============================] - 0s 53us/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 3/10
335/335 [==============================] - 0s 53us/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 4/10
335/335 [==============================] - 0s 49us/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 5/10
335/335 [==============================] - 0s 45us/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 6/10
335/335 [==============================] - 0s 61us/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 7/10
335/335 [==============================] - 0s 47us/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 8/10
335/335 [==============================] - 0s 48us/sample - loss: 2.4985 - mean_squared_error: 2.4985
Epoch 9/10
335/335 [==============================] - 0s 50us/sample - loss: 2.4985 - mean_square

## Ejecución de la primera red

Ejecutamos la primera red con los datos no categorizados

In [84]:
test_loss, test_acc = model_1.evaluate(X_test, y_test)

print('Mean Squared Error:', round(test_loss, 2))
#print('Test accuracy:', round(test_acc*100, 2), '%')

165/165 [==============================] - 0s 2ms/sample - loss: 2.2364 - mean_squared_error: 2.2364
Mean Squared Error: 2.24


## Entrenamiento de la segunda red

Entrenamos la segunda red usando los datos categorizados

In [85]:
X_train, X_test, y_train, y_test = categorical_train(x, y)

model_2 = keras.Sequential([
    keras.layers.Dense(1, input_dim=7, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
])

model_2.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_2.fit(X_train, y_train, epochs=15, batch_size=50, shuffle=False)
print(model_2.summary())

Epoch 1/15
335/335 [==============================] - 0s 1ms/sample - loss: 1.8350 - acc: 0.0657
Epoch 2/15
335/335 [==============================] - 0s 59us/sample - loss: 1.8079 - acc: 0.0657
Epoch 3/15
335/335 [==============================] - 0s 51us/sample - loss: 1.7823 - acc: 0.0687
Epoch 4/15
335/335 [==============================] - 0s 48us/sample - loss: 1.7582 - acc: 0.0657
Epoch 5/15
335/335 [==============================] - 0s 58us/sample - loss: 1.7354 - acc: 0.0597
Epoch 6/15
335/335 [==============================] - 0s 59us/sample - loss: 1.7141 - acc: 0.0746
Epoch 7/15
335/335 [==============================] - 0s 56us/sample - loss: 1.6942 - acc: 0.0866
Epoch 8/15
335/335 [==============================] - 0s 44us/sample - loss: 1.6759 - acc: 0.1075
Epoch 9/15
335/335 [==============================] - 0s 51us/sample - loss: 1.6594 - acc: 0.1343
Epoch 10/15
335/335 [==============================] - 0s 56us/sample - loss: 1.6445 - acc: 0.1582
Epoch 11/15
335/335 

## Ejecución de la segunda red

Ejecutamos la segunda red, usando los datos categorizados

In [86]:
test_loss, test_acc = model_2.evaluate(X_test, y_test)

print('Test accuracy:', round(test_acc*100, 2), '%')

165/165 [==============================] - 0s 2ms/sample - loss: 1.5918 - acc: 0.2909
Test accuracy: 29.09 %
